plan
- 没有学习过seq2seq
- 设计从多个指标通过mlp学习到一个x
- 从x预测天气
- 通过GRU之类的预测x

In [103]:
import pandas as pd
import numpy as np
seed=42
# 加载数据
df = pd.read_csv('/Users/Mac/code/Practice/ML/Kaggle/rnn_weather_pred/seattle-weather.csv')

# 将 'date' 列转换为 datetime 类型，方便后续处理
df['date'] = pd.to_datetime(df['date'])

def month_to_season(month):
    """将月份转换为季节：1=冬, 2=春, 3=夏, 4=秋"""
    if month in [12, 1, 2]:
        return 1  # Winter
    elif month in [3, 4, 5]:
        return 2  # Spring
    elif month in [6, 7, 8]:
        return 3  # Summer
    else:  # 9, 10, 11
        return 4  # Autumn

# 生成编码后的季节变量
df['season'] = df['date'].dt.month.apply(month_to_season)

# 天气编码：保留数值编码，同时保存编码与标签的对应关系
df['weather_code'], weather_labels = pd.factorize(df['weather'])

# 在内存中删除不需要的原始列：date 和 weather
df.drop(columns=['date', 'weather'], inplace=True)

# 将 weather_code 调整到最后面，season 放前面
front_cols = ['season']
other_cols = [c for c in df.columns if c not in front_cols + ['weather_code']]
df = df[front_cols + other_cols + ['weather_code']]

# 输出检查
print("天气类型编码（索引=编码，对应标签如下）:")
print(list(enumerate(weather_labels)))
print(df.head())

# 将处理好的数据保存到新的CSV文件中（不保存索引和表头）
df.to_csv('/Users/Mac/code/Practice/ML/Kaggle/rnn_weather_pred/seattle-weather-processed.csv', index=False, header=False)
print("数据已保存到 seattle-weather-processed.csv")

天气类型编码（索引=编码，对应标签如下）:
[(0, 'drizzle'), (1, 'rain'), (2, 'sun'), (3, 'snow'), (4, 'fog')]
   season  precipitation  temp_max  temp_min  wind  weather_code
0       1            0.0      12.8       5.0   4.7             0
1       1           10.9      10.6       2.8   4.5             1
2       1            0.8      11.7       7.2   2.3             1
3       1           20.3      12.2       5.6   4.7             1
4       1            1.3       8.9       2.8   6.1             1
数据已保存到 seattle-weather-processed.csv


In [104]:
data=np.loadtxt('seattle-weather-processed.csv',delimiter=',')
split=int(0.8*len(data))
x_train,y_train=data[:split,:5],data[:split,5].reshape(-1,1)
x_test,y_test=data[split:,:5],data[split:,5].reshape(-1,1)
# 转换为PyTorch张量
import torch

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

print(f"训练数据形状: x_train {x_train.shape}, y_train {y_train.shape}")
print(f"测试数据形状: x_test {x_test.shape}, y_test {y_test.shape}")




训练数据形状: x_train torch.Size([1168, 5]), y_train torch.Size([1168, 1])
测试数据形状: x_test torch.Size([293, 5]), y_test torch.Size([293, 1])


In [105]:

import torch.nn as nn
input_size=5
output_size=1
learning_rate=1e-3
hidden_1 = 16
hidden_2 = 16
max_epoch=500
num_classes=5
# mlp = nn.Sequential(
#     nn.Linear(input_size, hidden_1),
#     nn.ReLU(),
#     nn.Linear(hidden_1, hidden_2),
#     nn.ReLU(),
#     nn.Linear(hidden_2, output_size)
# )
# mlp_optim = torch.optim.Adam(mlp.parameters(), lr=learning_rate)
# for x,y in zip(x_train,y_train):

#定义为Encoder和decoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_1=32, x_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_1),
            nn.ReLU(),
            nn.Linear(hidden_1, x_dim),
            nn.ReLU(),  # 也可不加
        )
    def forward(self, x):
        return self.net(x)  # x_t

class Classifier(nn.Module):
    def __init__(self, x_dim, num_classes, hidden_dim_clf=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(x_dim, hidden_dim_clf),
            nn.ReLU(),
            nn.Linear(hidden_dim_clf, num_classes)
        )
    def forward(self, x_t):
        return self.net(x_t)

encoder = Encoder(input_size, hidden_1=32, x_dim=16)
clf = Classifier(16, num_classes)
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(clf.parameters()), lr=learning_rate)
acc = []
criterion = nn.CrossEntropyLoss()  # Assuming classification task

import matplotlib.pyplot as plt
from IPython.display import clear_output
for epoch in range(max_epoch):
    # 训练阶段
    encoder.train()
    clf.train()
    
    x_t = encoder(x_train)              # 编码得到潜在表示
    logits = clf(x_t)
    # 确保 y_train 是 1D 张量
    loss = criterion(logits, y_train.squeeze()) 
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 评估阶段
    with torch.inference_mode():
        encoder.eval()
        clf.eval()
        y_pred = clf(encoder(x_test))
        # 计算准确率并添加到acc列表
        _, predicted = torch.max(y_pred.data, 1)
        accuracy = (predicted == y_test.squeeze()).sum().item() / len(y_test)
        acc.append(accuracy)
    
    print(f'Epoch: {epoch+1}/{max_epoch}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')


Epoch: 1/500, Loss: 1.6907, Accuracy: 0.0000
Epoch: 2/500, Loss: 1.6610, Accuracy: 0.0068
Epoch: 3/500, Loss: 1.6327, Accuracy: 0.0137
Epoch: 4/500, Loss: 1.6056, Accuracy: 0.2491
Epoch: 5/500, Loss: 1.5797, Accuracy: 0.4334
Epoch: 6/500, Loss: 1.5548, Accuracy: 0.4608
Epoch: 7/500, Loss: 1.5306, Accuracy: 0.4608
Epoch: 8/500, Loss: 1.5070, Accuracy: 0.4676
Epoch: 9/500, Loss: 1.4838, Accuracy: 0.4812
Epoch: 10/500, Loss: 1.4606, Accuracy: 0.4881
Epoch: 11/500, Loss: 1.4374, Accuracy: 0.4915
Epoch: 12/500, Loss: 1.4143, Accuracy: 0.4983
Epoch: 13/500, Loss: 1.3913, Accuracy: 0.4983
Epoch: 14/500, Loss: 1.3684, Accuracy: 0.4983
Epoch: 15/500, Loss: 1.3458, Accuracy: 0.5222
Epoch: 16/500, Loss: 1.3236, Accuracy: 0.5495
Epoch: 17/500, Loss: 1.3018, Accuracy: 0.5631
Epoch: 18/500, Loss: 1.2806, Accuracy: 0.5700
Epoch: 19/500, Loss: 1.2603, Accuracy: 0.5768
Epoch: 20/500, Loss: 1.2407, Accuracy: 0.5836
Epoch: 21/500, Loss: 1.2218, Accuracy: 0.5904
Epoch: 22/500, Loss: 1.2035, Accuracy: 0.59

In [106]:
# --- 生成全部数据的 latent x ---
encoder.eval()
with torch.inference_mode():
    # 用训练好的编码器分别处理训练和测试数据
    x_train_encoded = encoder(x_train)  # 编码训练数据
    x_test_encoded = encoder(x_test)    # 编码测试数据

# def prepare_data(data,seq_len):
#     num_data=len(data)//(seq_len+1)*(seq_len+1)
#     data=data[:num_data]
#     x=data[:-1].reshape(_1,seq_len,1)
#     y=data[1:].reshape(_1,seq_len,1)
# 应该是转换为序列对


def prepare_data(data,seq_len):
    num_data=len(data)-seq_len
    x_seq=[]
    pre_seq=[]
    for i in range(num_data):
        x_seq.append(data[i:i+seq_len])
        pre_seq.append(data[i+1:i+seq_len+1])
    return torch.stack(x_seq),torch.stack(pre_seq)



class GRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers=1):
        super().__init__()
        self.hidden_size=hidden_size
        self.num_layers=num_layers

        self.gru=nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc=nn.Linear(hidden_size,output_size)

    def forward(self,x,h=None):
        if h is None:
            h = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)

        out,h=self.gru(x,h)
        pred=self.fc(out)
        return pred,h

    def predict(self,inital,pred_len):
        self.eval()
        with torch.no_grad():
            current_seq=inital.clone()
            h=None
            outputs=[]

            _,h=self.gru(current_seq,h)
            x_input=current_seq[:,-1:,:]

            for _ in range(pred_len):
                y_pred,h=self(x_input,h)
                outputs.append(y_pred)

                x_input=y_pred

            return torch.cat(outputs,dim=1)

def train_gru(model,x_train,y_train,epochs=100,lr=1e-3,batch_size=32):
        optimizer=torch.optim.Adam(model.parameters(),lr=lr)
        criterion=nn.MSELoss()

        if not isinstance(x_train,torch.Tensor):
            x_train=torch.FloatTensor(x_train)
            y_train=torch.FloatTensor(y_train)

        model.train()
        for epoch in range(epochs):
            indices=torch.randperm(len(x_train))
            total_loss=0
            
            for i in range(0,len(x_train),batch_size):
                batch_indices=indices[i:i+batch_size]
                x_batch=x_train[batch_indices]
                y_batch=y_train[batch_indices]

                y_pred,_=model(x_batch)
                loss=criterion(y_pred,y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss+=loss.item()

            if (epoch + 1) % 20 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(x_train):.6f}')
    
        return model

def evaluate_gru(model,x_test,y_test):
    model.eval()
    with torch.no_grad():
        if not isinstance(x_test,torch.Tensor):
            x_test=torch.FloatTensor(x_test)
            y_test=torch.FloatTensor(y_test)

        y_pred,_=model(x_test)
        mse=nn.MSELoss()(y_pred,y_test).item()

            # 计算预测准确率（如果是分类问题）
        if y_test.dim() > 2 and y_test.size(-1) > 1:  # 多分类情况
            pred_classes = torch.argmax(y_pred, dim=-1)
            true_classes = torch.argmax(y_test, dim=-1)
            accuracy = (pred_classes == true_classes).float().mean().item()
            print(f'Test MSE: {mse:.6f}, Accuracy: {accuracy:.4f}')
        else:  # 回归问题
            print(f'Test MSE: {mse:.6f}')

    return y_pred

In [107]:
seq_len=7

x_train,y_train=prepare_data(x_train_encoded,seq_len)
x_test,y_test=prepare_data(x_test_encoded,seq_len)

input_size=16
hidden_size=32
output_size=16

model=GRU(input_size,hidden_size,output_size)

model=train_gru(model,x_train,y_train,epochs=400)
y_pred=evaluate_gru(model,x_test,y_test)

initial_seq=torch.FloatTensor(x_test[:1])
future_preds=model.predict(initial_seq,pred_len=7)

print(future_preds.shape)


Epoch 20/400, Loss: 0.334533
Epoch 40/400, Loss: 0.304648
Epoch 60/400, Loss: 0.285047
Epoch 80/400, Loss: 0.245367
Epoch 100/400, Loss: 0.216428
Epoch 120/400, Loss: 0.200978
Epoch 140/400, Loss: 0.180620
Epoch 160/400, Loss: 0.166681
Epoch 180/400, Loss: 0.156438
Epoch 200/400, Loss: 0.150212
Epoch 220/400, Loss: 0.141974
Epoch 240/400, Loss: 0.140440
Epoch 260/400, Loss: 0.134800
Epoch 280/400, Loss: 0.130354
Epoch 300/400, Loss: 0.128276
Epoch 320/400, Loss: 0.124708
Epoch 340/400, Loss: 0.122075
Epoch 360/400, Loss: 0.119145
Epoch 380/400, Loss: 0.116920
Epoch 400/400, Loss: 0.117597
Test MSE: 16.996431, Accuracy: 0.7897
torch.Size([1, 7, 16])
